In [ ]:
# Install the missing packages
! pip install tensorflow-addons
! pip install gdown
! pip install zipfile

# Import the libraries needed
import os
import numpy as np
from   PIL import Image
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import tensorflow_addons as tfa
import gdown
from   tensorflow.keras.preprocessing import image_dataset_from_directory
from   tensorflow.keras.applications import EfficientNetB4

# Define some shortcuts
tfk  = tf.keras
tfkl = tf.keras.layers

# Download and unzip the dataset
! gdown --id 1mZqRi_NjuozRQdO3CzbKS4ve-erTRWuX
! unzip dataset.zip

# Download and unzip the image dataset
! gdown --id 179XBNS5FU1dAZTOMqsA1_e1xgiGhlsxl
! tar -xf noisy_student_efficientnet-b4.tar.gz

# Download the noisy student efficientnet-b4 weights (state of the art pre-trained weights for this model). Convert them into a '.h5' model
! gdown --id 1nnHTLskmwOyKWgEk7wcShKLSZ23eAFTH
! python efficientnet_weight_update_util.py --model b4 --notop --ckpt ./noisy_student_efficientnet-b4/model.ckpt --o efficientnetb4_notop.h5

# Definition and application of the random seed for reproducibility
seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
# Definition of the training folder, it will be useful later
training_dir = 'training'

# Plot example images from dataset
labels_list = ['Apple', 'Blueberry', 'Cherry', 'Corn', 'Grape', 'Orange', 'Peach', 'Pepper', 'Potato', 'Raspberry', 'Soybean', 'Squash', 'Strawberry', 'Tomato']
num_row     = len(labels_list)//2
num_col     = len(labels_list)//num_row
fig, axes   = plt.subplots(num_row, num_col, figsize=(2*num_row,15*num_col))

for i in range(len(labels_list)):
  if i < len(labels_list):
    class_imgs = next(os.walk('{}/{}/'.format(training_dir, labels_list[i])))[2]
    class_img  = class_imgs[0]
    img        = Image.open('{}/{}/{}'.format(training_dir, labels_list[i], class_img))
    ax         = axes[i//num_col, i%num_col]
    ax.imshow(np.array(img))
    ax.set_title('{}'.format(labels_list[i]))
    
plt.tight_layout()
plt.show()

In [ ]:
# Generation of a tf.data.Dataset from image files in a directory (both for training and validation)

train_dataset = image_dataset_from_directory(
                    directory        = training_dir,
                    labels           = 'inferred',
                    label_mode       = 'categorical',
                    class_names      = labels_list,
                    color_mode       = 'rgb',
                    batch_size       = 64,
                    image_size       = (256, 256),
                    shuffle          = True,
                    seed             = seed,
                    validation_split = 0.2,
                    subset           = 'training')

val_dataset = image_dataset_from_directory(
                    directory        = training_dir,
                    labels           = 'inferred',
                    label_mode       = 'categorical',
                    class_names      = labels_list,
                    color_mode       = 'rgb',
                    batch_size       = 64,
                    image_size       = (256, 256),
                    shuffle          = True,
                    seed             = seed,
                    validation_split = 0.2,
                    subset           = 'validation')

# Buffered prefetching allows us to yield data from disk without having I/O becoming a bottleneck.
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE)
val_dataset   = val_dataset.prefetch(tf.data.AUTOTUNE)

# Performing data augmentation inside the model improves dramatically the performances. From keras documentation:
# With this option, preprocessing will happen on device, synchronously with the rest of the model execution, meaning that it will benefit from GPU acceleration

data_augmentation = tfk.Sequential(
                            [
                            tfkl.RandomFlip(mode = 'horizontal_and_vertical', seed = seed),
                            tfkl.RandomTranslation(height_factor = (-0.5, 0.5), width_factor = (-0.5, 0.5), fill_mode = 'reflect', seed = seed), 
                            tfkl.RandomRotation(factor = 0.3, fill_mode = 'reflect', seed = seed),
                            tfkl.RandomZoom(height_factor = (-0.3, 0.3), fill_mode = 'reflect', seed = seed), 
                            tfkl.RandomContrast(factor = 0.3, seed = seed)
                            ]
                            )

In [ ]:
# Model metadata
input_shape = (256, 256, 3)
epochs = 200

# CNN model
# Preprocessing + (Conv + ReLU + BatchNorm + MaxPool) x 5 + FC x 2
def build_model(input_shape):
    """
    build_model generates the neural network model for the given input shape.
    The model is made up by the following layers
    - Input layer
    - Augmentation layer
    - EfficientNetB4 layers (without the dense part at the top)
    - GlobalAveragePooling2D layer
    - BatchNormalization layer
    - Dense layers 
    """
    
    input_layer               = tfkl.Input(shape = input_shape)
    augmentation_layer        = data_augmentation(input_layer)
    # noise_layer             = tfkl.GaussianNoise(stddev = 0.1)(augmentation_layer)
    efficient_layer           = EfficientNetB4(weights = "efficientnetb4_notop.h5", include_top = False, input_tensor = augmentation_layer)
    # Since we are performing transfer learning, the efficientnet layers are set to be non-trainable
    efficient_layer.trainable = False
    x                         = tfkl.GlobalAveragePooling2D()(efficient_layer.output)
    x                         = tfkl.BatchNormalization()(x)
    classifier_layer          = tfkl.Dense(units = 512, kernel_initializer = tfk.initializers.GlorotUniform(seed), activation = 'relu')(x)
    classifier_layer          = tfkl.Dropout(0.3, seed = seed)(classifier_layer)
    output_layer              = tfkl.Dense(units = 14, activation = 'softmax', kernel_initializer = tfk.initializers.GlorotUniform(seed))(classifier_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'model')
    
    # clr = tfa.optimizers.Triangular2CyclicalLearningRate(initial_learning_rate = 1e-4,
    #                                                   maximal_learning_rate = 6e-2,
    #                                                   step_size = 2 * 222)
    
    #f1 = tfa.metrics.F1Score(num_classes = 14, average = 'micro')
    
    # Compile the model
    model.compile(loss      = tfk.losses.CategoricalCrossentropy(), 
                  optimizer = tfk.optimizers.Adam(learning_rate = 1e-2), 
                  metrics   = ['accuracy']
                 )
    
    # Return the model
    return model

from datetime import datetime

def create_folders_and_callbacks(model_name):
    """
    Callbacks definition: Save model checkpoints and EarlyStopping
    """

    exps_dir = os.path.join('data_augmentation_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)

    now = datetime.now().strftime('%b%d_%H-%M-%S')

    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
      
    callbacks = []

    # Model checkpoint
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)

    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath          = os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only = False, # True to save only weights
                                                     save_best_only    = True) # True to save only the best epoch 
    callbacks.append(ckpt_callback)

    # Early Stopping
    es_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 10, restore_best_weights = True)
    callbacks.append(es_callback)

    return callbacks

# Build the model
model = build_model(input_shape)

# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name = 'CNN_Aug_Transfer_Learning')

# # Train the model
history = model.fit(
     x               = train_dataset,
     epochs          = epochs,
     validation_data = val_dataset,
     callbacks       = callbacks).history

# Save best epoch model
model.save("data_augmentation_experiments/CNN_Aug_Transfer_Learning_Best")

# Plot the training
plt.figure(figsize = (15, 5))
plt.plot(history['loss'], label = 'Training', alpha = .8, color = '#ff7f0e')
plt.plot(history['val_loss'], label = 'Validation', alpha = .8, color = '#4D61E2')
plt.legend(loc = 'upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha = .3)

plt.figure(figsize = (15,5))
plt.plot(history['accuracy'], label = 'Training', alpha = .8, color = '#ff7f0e')
plt.plot(history['val_accuracy'], label = 'Validation', alpha = .8, color = '#4D61E2')
plt.legend(loc = 'upper left')
plt.title('Accuracy')
plt.grid(alpha = .3)

plt.show()

In [ ]:
# EfficientNet network is made up by 7 blocks. During fine tuning, it is suggested to unfreeze whole layers inside a block. In the following lines of code the
# 7th block gets complitely unfrozen, except fot the BatchNormalization layers, as suggested in the Keras documentation

for layer in model.layers[-36:]:
    if not isinstance(layer, tfkl.BatchNormalization):
        layer.trainable = True

for layer in model.layers:
    print("{}: {}".format(layer, layer.trainable))  
    
# Compile the model
model.compile(loss      = tfk.losses.CategoricalCrossentropy(), 
              optimizer = tfk.optimizers.Adam(learning_rate=1e-4), 
              metrics   = ['accuracy'])

# Create folders and callbacks and fit
callbacks = create_folders_and_callbacks(model_name = 'CNN_Aug_Fine_Tuning')

#reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', factor = 0.2, patience = 5, min_lr = 1e-6)
#aug_callbacks.append(reduce_lr_callback)

# Train the model
history_fine_tuning = model.fit(
     x = train_dataset,
     epochs = epochs,
     validation_data = val_dataset,
     callbacks = callbacks).history

# Save best epoch model
model.save("data_augmentation_experiments/CNN_Aug_Fine_Tuning_Best")

# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history_fine_tuning['loss'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history_fine_tuning['val_loss'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history_fine_tuning['accuracy'], label='Training', alpha=.8, color='#ff7f0e')
plt.plot(history_fine_tuning['val_accuracy'], label='Validation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()


In [ ]:

# Zip and download the work folder for the submission

!zip -r fine_tuning.zip /kaggle/working